In [ ]:
from dask.distributed import LocalCluster, Client
from dask import delayed, compute
from collections import OrderedDict

import dask.dataframe as dd
import fastparquet as fq

import multiprocessing as mp
import pandas as pd
import numpy as np
import time

In [ ]:
! ls -l data
to_read = 'data/' +  input('copy and paste the filename: ')

In [ ]:
data = fq.ParquetFile(to_read).to_pandas()

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
programs = list(data.program.unique())
programs.sort()
programs = pd.Series(programs)

In [ ]:
q = input('search programs:')
mask = programs.str.startswith(q)
programs.loc[mask]

In [ ]:
to_match = OrderedDict(year='2017', month='02', program='GOOD MORNING AMERICA', demo='WM2554', time='07:00-09:00', mon=1, tue=1, wed=1, thur=1, fri=1)
#to_match = OrderedDict(program='MORNING NEWS', time='06:00-07:00', station_affiliation='CBS')

The following section compares query runtime with and without reduction. Building a chain of loc slows things down severely. The best time was without reduction.

In [ ]:
%%timeit
frame, mask = data, np.ones(len(data)).astype(bool)
for k, v in to_match.items():
    t1 = time.time()
    #mask &= frame.loc[mask, k] == v
    mask &= data[k] == v
    frame = frame.loc[mask]
    t2 = time.time()
    #print(k, t2-t1)
y = frame.imp.sum()
print(y)

In [ ]:
%%timeit
mask = np.ones(len(data)).astype(bool)
for k, v in to_match.items():
    t1 = time.time()
    #mask &= data.loc[mask, k] == v
    mask &= data[k] == v
    t2 = time.time()
    #print(k, t2-t1)
frame = data.loc[mask]
y = frame.imp.sum()
print(y)

time to daskify!

In [ ]:
cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
df = dd.from_pandas(data, npartitions=mp.cpu_count())

In [ ]:
%%time
ddata = client.persist(df)

In [ ]:
#masks = [ddata.loc[:, k] == v for k, v in to_match.items()]
masks = [ddata[k] == v for k, v in to_match.items()]

In [ ]:
mask = masks[0]
for m in masks[1:]:
    mask &= m

In [ ]:
y = ddata.loc[mask, 'imp'].sum()

In [ ]:
%%timeit
yy = y.compute()

In [ ]:
yy

manual sanity check and loc investigtaion

In [ ]:
%%timeit
m = data.loc[:, 'year'] == '2017'
m &= data.loc[m, 'month'] == '02'
m &= data.loc[m, 'demo'] == to_match['demo']
m &= data.loc[m, 'program'] == to_match['program']
m &= data.loc[m, 'time'] == '07:00-09:00'
data[m].imp.sum()

In [ ]:
%%timeit
m = data.year == '2017'
m &= data.month == '02'
m &= data.demo == to_match['demo']
m &= data.program == to_match['program']
m &= data.time == '07:00-09:00'
data[m].imp.sum()

let's investigate how to optimize queries manually

In [ ]:
clabels = [l for l, t in data.dtypes.items() if str(t) == 'category']
d = {label: len(data[label].cat.categories) for label in clabels}
s = pd.Series.from_array(d)

In [ ]:
s.sort_values(ascending=False, inplace=True)

In [ ]:
s

In [ ]:
%%timeit
m = True
m &= data.program == to_match['program']
m &= data.time == '07:00-09:00'
m &= data.demo == to_match['demo']
m &= data.month == '02'
m &= data.year == '2017'
data[m].imp.sum()

In [ ]:
%%timeit
f = data
m = f.program == to_match['program']
f = f[m]
m = f.time == '07:00-09:00'
f = f[m]
m = f.demo == to_match['demo']
f = f[m]
m = f.month == '02'
f = f[m]
m = f.year == '2017'
f = f[m]
f.imp.sum()

let's use built in pandas query engine

In [ ]:
qstr = ' and '.join([f'{k}=="{v}"' for k, v in to_match.items()])

In [ ]:
qstr

In [ ]:
%%time
r = data.query(qstr)
r = r.imp.sum()
print(r)

In [ ]:
%%time
y = ddata.query(qstr).imp.sum()
yy = y.compute()

In [ ]:
y.visualize()

In [ ]:
yy